# Textiles Mexicanos

## 1. Obtención de datos
Obtenemos el dataset del siguiente enlace https://www.mediafire.com/file/xgjtpj9pxbaly2q/UnlabeledData.zip/file, esto se hace utilizando web scraping con las librerías lxml y requests, termcolor sólo se utiliza para dar color en la salida. La instalación de las librerías se realiza de la siguiente forma. 

```sh
pip install lxml requests termcolor
```

Para evitar ser baneados, declaramos un diccionario headers, que contrendrá algunos atributos que la página leerá para autenticar la petición.

```python
headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64; rv:103.0) Gecko/20100101 Firefox/103.0",
}
```

In [ ]:
import requests
from lxml import html
from termcolor import cprint

headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64; rv:103.0) Gecko/20100101 Firefox/103.0",
}

Se declara una función que descarga un archivo estático de cualquier url, por ejemplo https://www.ipn.mx/assets/files/main/img/template/logo_ipn_guinda.svg se podría descargar con la siguiente función.

```python
download_file(
    url="https://www.ipn.mx/assets/files/main/img/template/logo_ipn_guinda.svg",
    file_name="ipn.svg",
    headers=default_headers
)
```

La ejecución de esta función daría como resultado la descarga de este archivo en la ruta actual.

In [ ]:
def download_file(url, file_name="descarga", headers=headers):
    if not file_name:
        file_name = url.split("/").pop()

    file = requests.get(url, headers=headers)
    downloaded = open(file_name, "wb")
    downloaded.write(file.content)
    downloaded.close()

Combinando todo lo anterior ya podemos descargar el dataset completo de mediafire y en general, cualquier archivo que se aloje dentro de esta página.

In [ ]:
def mediafire(url, headers):
    cprint(f"Downloading from {url}")
    response = requests.get(url, headers)
    html_text = response.text
    parser = html.fromstring(html_text)
    file_link = parser.xpath('//a[@class="input popsok"]/@href')

    cprint(f"Downloading from {url}", "magenta")
    link = file_link[0]
    file_name = file_link[0].split("/").pop()
    download_file(link, file_name, headers)
    cprint(f"file {file_name} Downloaded", "green")

Se descarga el dataset

In [ ]:
data_url = "https://www.mediafire.com/file/xgjtpj9pxbaly2q/UnlabeledData.zip/file"
mediafire(data_url,headers)

Se descomprime y se guarda en la carpeta data

In [ ]:
!unzip UnlabeledData.zip
!rm UnlabeledData.zip
!mv UnlabeledData data

## 2. Preprocesamiento
La idea es trasladar una imagen a color en un vector de características que represente fielmente la información de la imagen. Importamos nuestras librerías básicas. Tenemos 2 opciones, generar las características o cargarlas de un archivo csv.

In [8]:
import pandas as pd
import numpy as np
import cv2
import os

import matplotlib.pyplot as plt

### 2.1 Obtención de características

**1 Creación de características**

Para obtener las características todas las imágenes ingresarán en una red neuronal que para imágenes similares, dará vectores de características similares.

In [ ]:
# Carga y preprocesamiento de Imágenes
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img ,img_to_array
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input

# Model preenetrenado de extracción de características de la imagen 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

Se instancia el modelo y truncamos la última capa para evitar que nos de clasificaciones de la imagen

In [ ]:
model = VGG16()
model = Model(
    inputs=model.inputs, 
    outputs=model.layers[-2].output
)

Para automatizar este proceso, redimensionamos la imagen a 224 x 224 píxeles y esta se conserva a color, hacemos la predicción con el modelo y obtenemos un vector de características de 4096 componentes.

In [5]:
def get_features(image_path,model):
  img = load_img(image_path, target_size=(224, 224))
  img = np.array(img)

  reshaped_img = img.reshape(1,224,224,3)
  x = preprocess_input(reshaped_img)

  features = model.predict(x)
  return features

Realizamos la predicción para todos los archivos de imágen que se encuentren el el directorio `data`, esto se guarda en la variable dataset.

In [6]:
datadir = "data"
files = os.listdir(datadir)

dataset = [get_features(os.path.join(datadir,file),model) for file in files]
dataset  = [vector[0] for vector in dataset]

Una vez se cargan los datos guardamos las características en un archivo csv, donde habrá 4096 columnas y varias filas, una simple tabla que contendrá toda la representación de las imágenes. Al menos mientras no se borre ningún archivo, pues todo depende totalmente del orden de los archivos, una mínima alteración en el orden arruinará todo.

In [11]:
labels = [f"x{i}" for i in range(0,4096)]
df = pd.DataFrame(data=dataset,columns=labels)
df.to_csv("Features.csv")

**2 Carga desde un archivo CSV**

Para evitar todo el procesamiento anterior cada vez que se corra el notebook, mejor se carga desde un archivo csv.

In [9]:
datadir = "data"
dataset = pd.read_csv("db/Features.csv")
dataset = dataset.iloc[:,:].values 
dataset[0]

array([0.       , 0.       , 1.8900677, ..., 0.       , 0.381624 ,
       0.       ])

### 2.2 Reducción de Dimensionalidad

Se utilizará KPCA puesto que es un problema no lineal, sobre el número de componentes, se ha elegido 20 para no complicar la selección hacia atrás usada en la regresión lineal múltiple.

In [10]:
from sklearn.decomposition import KernelPCA

components = 20
kpca = KernelPCA(n_components=components,kernel="rbf")
red_data =  kpca.fit_transform(dataset)

red_data[:5]

array([[ 0.00337295, -0.06090929,  0.04436981, -0.07733339, -0.12020049,
         0.04915178, -0.09281499, -0.12902319,  0.20866329, -0.09260902,
        -0.14523923, -0.02626722, -0.13717878,  0.06608574,  0.04010848,
        -0.02227136,  0.02576424, -0.04034996, -0.02892124,  0.0150692 ],
       [ 0.00053323, -0.05006775,  0.03643293, -0.06397324, -0.09596588,
         0.04327814, -0.07450606, -0.10547256,  0.16550379, -0.08068934,
        -0.10362385, -0.02915939, -0.11921509,  0.05464905,  0.02923226,
        -0.02347757,  0.01792858, -0.02179589, -0.01021651,  0.00256689],
       [ 0.00297774, -0.05766386,  0.04147192, -0.07186964, -0.10992257,
         0.04550583, -0.08263484, -0.11413934,  0.18068463, -0.08277731,
        -0.11921536, -0.02255266, -0.11882027,  0.05382289,  0.02884547,
        -0.03151722,  0.02116779, -0.02858183, -0.01654577,  0.00731569],
       [-0.00248787, -0.03879777,  0.02839421, -0.05062358, -0.07212567,
         0.03752514, -0.05701817, -0.08273231,  

Guardamos el conjunto de datos reducido en un nuevo dataset

In [ ]:
labels = [f"x{i}" for i in range(components)]
df_red = pd.DataFrame(data=red_data,columns=labels)
df_red.to_csv("Reduced.csv")

## 3 Procesamiento
Se probarán distintos métodos de agrupamiento como K-Means y Clustering Jerárquico

### 3.1 K-Means

Se realiza el método del codo para seleccionar el número adecuado de clusters

In [ ]:
from sklearn.cluster import KMeans

sse = []
list_k = list(range(2, 12))

for k in list_k:
    km = KMeans(n_clusters=k, random_state=22)
    km.fit(red_data)
    sse.append(km.inertia_)

plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance')

Entrenamos el modelo de K-Means, sólo se puede entrenar uno a la vez

In [ ]:
nclusters = 6
model = KMeans(n_clusters=nclusters, random_state=22)
model.fit(red_data)
labels = model.labels_

### 3.2 Clustering Jerárquico
Se realiza el árbol de endobrama

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

linkage_data = linkage(dataset, method='ward', metric='euclidean')

dendrogram(linkage_data)
plt.show()

Se han obtenido 5 clusters

In [ ]:
hierarchical_cluster = AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='ward')
labels = hierarchical_cluster.fit_predict(dataset) 

### 3.3 Resultados del agrupamiento
Se guardan los datos donde una columna es el archivo de la imagen y la otra es el grupo al que pertenece

In [ ]:
df = {
    "image" : os.listdir("data"),
    "label" : labels
}

df = pd.DataFrame(df)
df.to_csv("data.csv")
df

La siguiente función permite graficar un cluster completo

In [27]:
def view_cluster(cluster,mean):
    clusterFiles = df[df['label'] == cluster ]
    files = clusterFiles["image"].values[0:10]
    labels = clusterFiles["label"].values[0:10]
    images = [cv2.imread(os.path.join(datadir,file)) for file in files]

    fig = plt.figure(figsize=(50, 11))
    fig.suptitle(f"Cluster {cluster} : {mean}",size=100)

    rows = 1
    columns = 10

    for index,image in enumerate(images):
      fig.add_subplot(rows, columns, index+1)
      plt.imshow(image)
      plt.axis('off')
      plt.title(f"{str(index)}")

Resultados de K-Means

In [ ]:
means = ["Zapoteco","Purepecha","Chinanteco","Mixe","Huichol","Otomi"]

for index,mean in enumerate(means):
  view_cluster(index,means[index])

Resultados de clustering jerárquico

In [ ]:
means = ["Zapoteco","Purepecha","Chinanteco","Mixe","Huichol"]

for index,mean in enumerate(means):
  view_cluster(index,means[index])

## 4 Evaluación
Se usará el índice de dunn y Davis Bouldin, el primero se instalará desde una librería externa y el segundo ya viene incluido en sklearn.


### 4.1 Indice de Dunn

In [ ]:
!git clone https://gitlab.com/jqmviegas/jqm_cvi jqm_cvi
!cd jqm_cvi && mv jqmcvi ..
!cd ..
!rm -rf jqm_cvi

In [ ]:
# Indice de Dunn
from jqmcvi import base

dun = base.dunn_fast(red_data,model.labels_)
print(f"Indice de Dunn {dun}")

### 4.2 Indice de Davies Bouldin

In [ ]:
from sklearn.metrics import davies_bouldin_score

DB = davies_bouldin_score(red_data,model.labels_)
print(f"Indice de Davies Bouldin : {DB}")